In [1]:
from IPython.display import Markdown, Video
with open('README.md') as f:
    display(Markdown('\n'.join(f.readlines())))

# Minimal embeddings example

## with TensorFlow and the iris dataset

#### by August Karlstedt

todo: write




In [2]:
import glob

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)
np.random.seed(1234)

In [3]:
file = 'data/bezdekIris.data'

data = pd.read_csv(file, names=['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])
data['sepal length'] = data['sepal length'].astype(np.float32)
data['sepal width'] = data['sepal width'].astype(np.float32)
data['petal length'] = data['petal length'].astype(np.float32)
data['petal width'] = data['petal width'].astype(np.float32)
data.describe().T

,count,mean,std,min,25%,50%,75%,max
sepal length,150.0,5.843333,0.828066,4.3,5.1,5.80,6.4,7.9
sepal width,150.0,3.057333,0.435866,2.0,2.8,3.00,3.3,4.4
petal length,150.0,3.758000,1.765298,1.0,1.6,4.35,5.1,6.9
petal width,150.0,1.199333,0.762238,0.1,0.3,1.30,1.8,2.5


Our values match what is provided in `data/iris.names`

In [4]:
data.sample(10)

,sepal length,sepal width,petal length,petal width,class
91,6.1,3.0,4.6,1.4,Iris-versicolor
63,6.1,2.9,4.7,1.4,Iris-versicolor
103,6.3,2.9,5.6,1.8,Iris-virginica
6,4.6,3.4,1.4,0.3,Iris-setosa
59,5.2,2.7,3.9,1.4,Iris-versicolor
29,4.7,3.2,1.6,0.2,Iris-setosa
27,5.2,3.5,1.5,0.2,Iris-setosa
35,5.0,3.2,1.2,0.2,Iris-setosa
99,5.7,2.8,4.1,1.3,Iris-versicolor
122,7.7,2.8,6.7,2.0,Iris-virginica


In [5]:
x_train = data[['sepal length', 'sepal width', 'petal length', 'petal width']]
x_train = x_train.values

In [6]:
unique_classes = data['class'].unique().tolist()
unique_classes

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

In [7]:
class_count = len(unique_classes)
class_count

3

In [8]:
class_to_int = {}
for i, item in enumerate(unique_classes):
    class_to_int[item] = i
class_to_int

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [9]:
def int_to_class(i):
    return unique_classes[i]
int_to_class(1)

'Iris-versicolor'

In [10]:
y_index = []
y_onehot = []

for item in data[['class']].values:
    item = item[0]
    
    vec = np.zeros(class_count)
    index = class_to_int[item]
    vec[index] = 1
    
    y_index.append(index)
    y_onehot.append(vec)

data['index'] = y_index
data['onehot'] = y_onehot

In [11]:
data.sample(10)

,sepal length,sepal width,petal length,petal width,class,index,onehot
76,6.8,2.8,4.8,1.4,Iris-versicolor,1,"[0.0, 1.0, 0.0]"
105,7.6,3.0,6.6,2.1,Iris-virginica,2,"[0.0, 0.0, 1.0]"
73,6.1,2.8,4.7,1.2,Iris-versicolor,1,"[0.0, 1.0, 0.0]"
117,7.7,3.8,6.7,2.2,Iris-virginica,2,"[0.0, 0.0, 1.0]"
115,6.4,3.2,5.3,2.3,Iris-virginica,2,"[0.0, 0.0, 1.0]"
128,6.4,2.8,5.6,2.1,Iris-virginica,2,"[0.0, 0.0, 1.0]"
15,5.7,4.4,1.5,0.4,Iris-setosa,0,"[1.0, 0.0, 0.0]"
18,5.7,3.8,1.7,0.3,Iris-setosa,0,"[1.0, 0.0, 0.0]"
55,5.7,2.8,4.5,1.3,Iris-versicolor,1,"[0.0, 1.0, 0.0]"
77,6.7,3.0,5.0,1.7,Iris-versicolor,1,"[0.0, 1.0, 0.0]"


In [12]:
y_train = np.array(y_onehot)

In [13]:
NB_EPOCHS = 1024
BATCH_SIZE = 16
LEARNING_RATE = 0.0001

In [14]:
sess = tf.Session()

In [15]:
x = tf.placeholder(tf.float32, shape=(None, 4))
y = tf.placeholder(tf.float32, shape=(None, class_count))

out = tf.layers.Dense(3, activation='linear', kernel_initializer='uniform', use_bias=False)(x)
logits = tf.layers.Dense(class_count)(out)
probs = tf.nn.softmax(logits=logits)

rng = np.random.RandomState([2019, 10, 20])

W1022 15:17:06.679523  5476 deprecation.py:506] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1022 15:17:06.881071  5476 deprecation.py:506] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
loss = tf.losses.softmax_cross_entropy(y, logits)

W1022 15:17:06.925168  5476 deprecation.py:323] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\losses\losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
opt = tf.train.GradientDescentOptimizer(LEARNING_RATE)

In [18]:
train = opt.minimize(loss)

In [19]:
sess.run(tf.global_variables_initializer())

In [20]:
fig = plt.figure()

<Figure size 432x288 with 0 Axes>

In [21]:
for epoch in range(NB_EPOCHS):
    print('Epoch', epoch, ' ', end='')
    
    data_x = x_train
    data_y = y_train
    
    r1, r2, r3, r4 = sess.run([train, loss, probs, out], feed_dict={x: data_x, y: data_y})
    print('Loss', r2, ' ', end='')
    acc = 0
    tot = 0
    for i, j in enumerate(np.argmax(r3, axis=1)):
        vec = np.zeros(class_count)
        vec[j] = 1.0
        if np.allclose(data_y[i], vec):
            acc += 1
        tot += 1
    print('Acc', acc/tot, ' ')
    
    pos = r4
    col = np.argmax(data_y, axis=1)
    fig.clf()
    ax = fig.add_subplot(111, projection='3d')
    ax.elev = 15.0
    ax.azim = 15.0
    ax.scatter(pos[:, 0], pos[:, 1], pos[:, 2], c=col, cmap='Set1')
    fig.savefig(f'frames/{epoch}.png')
    
    while data_x.any():
        bs = BATCH_SIZE
        if data_x.shape[0] < BATCH_SIZE:
            bs = data_x.shape[0]
        
        indices = np.arange(0, data_x.shape[0], step=1)
        batch_indices = np.random.choice(indices, size=bs, replace=False)

        batch_x = data_x[batch_indices]
        batch_y = data_y[batch_indices]
            
        sess.run(train, feed_dict={x: batch_x, y: batch_y})
        
        data_x = np.delete(data_x, batch_indices, axis=0)
        data_y = np.delete(data_y, batch_indices, axis=0)
        indices = np.delete(indices, batch_indices, axis=0)

Epoch 0  Loss 1.1175056  Acc 0.3333333333333333  
Epoch 1  Loss 1.1162529  Acc 0.3333333333333333  
Epoch 2  Loss 1.1150053  Acc 0.3333333333333333  
Epoch 3  Loss 1.1136588  Acc 0.3333333333333333  
Epoch 4  Loss 1.112493  Acc 0.3333333333333333  
Epoch 5  Loss 1.111351  Acc 0.3333333333333333  
Epoch 6  Loss 1.1100862  Acc 0.3333333333333333  
Epoch 7  Loss 1.1089019  Acc 0.3333333333333333  
Epoch 8  Loss 1.1076524  Acc 0.32666666666666666  
Epoch 9  Loss 1.1064526  Acc 0.32666666666666666  
Epoch 10  Loss 1.1054059  Acc 0.32666666666666666  
Epoch 11  Loss 1.1044073  Acc 0.29333333333333333  
Epoch 12  Loss 1.1034153  Acc 0.29333333333333333  
Epoch 13  Loss 1.1023979  Acc 0.24  
Epoch 14  Loss 1.1014053  Acc 0.20666666666666667  
Epoch 15  Loss 1.1003796  Acc 0.13333333333333333  
Epoch 16  Loss 1.099408  Acc 0.08666666666666667  
Epoch 17  Loss 1.0984799  Acc 0.06  
Epoch 18  Loss 1.0975082  Acc 0.03333333333333333  
Epoch 19  Loss 1.0966029  Acc 0.013333333333333334  
Epoch 20  

In [22]:
#C:\Users\August\Downloads\ffmpeg.exe -i %d.png out.webm

In [29]:
display(Video('./frames/out.webm'))

In [24]:
random_example = data.sample(1)
random_example

,sepal length,sepal width,petal length,petal width,class,index,onehot
8,4.4,2.9,1.4,0.2,Iris-setosa,0,"[1.0, 0.0, 0.0]"


In [25]:
example_x = random_example[['sepal length', 'sepal width', 'petal length', 'petal width']]
example_x

,sepal length,sepal width,petal length,petal width
8,4.4,2.9,1.4,0.2


In [26]:
example_y = random_example['onehot']
example_y

8    [1.0, 0.0, 0.0]
Name: onehot, dtype: object

In [27]:
p = sess.run(probs, feed_dict={x: example_x })
p

array([[0.5216935 , 0.30448022, 0.17382617]], dtype=float32)

In [28]:
int_to_class(p.argmax())

'Iris-setosa'